In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
response = requests.get('https://chatgpt.com/share/678a02ef-5848-8006-b26e-df43cd91d35e')

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
body = soup.find('body')
scripts = body.find_all('script')

scripts = [(len(script.text), script) for script in scripts]
scripts.sort(reverse=True)
json_obj_match = re.search(r'window\.__remixContext\s*=\s*(\{.*?\});__remixContext', scripts[0][1].text, re.DOTALL)
# json_obj_match = re.search(r'window\.__remixContext\s*=\s*(\{[\s\S]*\});', scripts[0][1].text, re.DOTALL)

json_obj_str = json_obj_match.group(1)
json_obj = json.loads(json_obj_str)
with open("tmp.json", 'w') as f:
    json.dump(json_obj, f, indent=4)

In [4]:
scripts[0][1].text

'window.__remixContext = {"basename":"/","future":{"v3_fetcherPersist":false,"v3_relativeSplatPath":false,"v3_throwAbortReason":false,"unstable_singleFetch":false,"unstable_lazyRouteDiscovery":false,"unstable_optimizeDeps":true},"isSpaMode":false,"state":{"loaderData":{"root":{"rq:[\\"session\\"]":{"data":null,"dataUpdateCount":1,"dataUpdatedAt":1737433322831,"error":null,"errorUpdateCount":0,"errorUpdatedAt":0,"fetchFailureCount":0,"fetchFailureReason":null,"fetchMeta":null,"isInvalidated":false,"status":"success","fetchStatus":"idle"},"rq:[\\"account-status\\"]":{},"clientBootstrap":{"session":{},"user":{"email":"","groups":[],"id":"ua-2006a131-8671-4523-b6c1-7916733fbaa7","name":"","intercom_hash":"ua-2006a131-8671-4523-b6c1-7916733fbaa7","image":"","picture":"","mfa":false},"cluster":"unified-5","userCountry":"US","userRegion":"California","userRegionCode":"CA","cfConnectingIp":"68.181.218.10","cfIpLatitude":"34.03220","cfIpLongitude":"-118.28360","cfTimezone":"America/Los_Angeles"

In [5]:
json_obj['state']['loaderData']['routes/share.$shareId.($action)']['serverResponse']['data']['title']

'PagedAttention Paper Summary'

In [6]:
json_obj['state']['loaderData']['routes/share.$shareId.($action)']['serverResponse']['data']

{'title': 'PagedAttention Paper Summary',
 'create_time': 1737097967.345798,
 'update_time': 1737097969.906868,
 'mapping': {'aaa1ad37-9645-4125-b616-26c1ffd346c2': {'id': 'aaa1ad37-9645-4125-b616-26c1ffd346c2',
   'children': ['34b5887a-7c3a-44e8-a3c7-2b1ce9873e84']},
  '34b5887a-7c3a-44e8-a3c7-2b1ce9873e84': {'id': '34b5887a-7c3a-44e8-a3c7-2b1ce9873e84',
   'message': {'id': '34b5887a-7c3a-44e8-a3c7-2b1ce9873e84',
    'author': {'role': 'system', 'metadata': {}},
    'content': {'content_type': 'text', 'parts': ['']},
    'status': 'finished_successfully',
    'end_turn': True,
    'weight': 0,
    'metadata': {'is_visually_hidden_from_conversation': True,
     'shared_conversation_id': '678a02ef-5848-8006-b26e-df43cd91d35e'},
    'recipient': 'all'},
   'parent': 'aaa1ad37-9645-4125-b616-26c1ffd346c2',
   'children': ['9e3cd8a4-6c03-4f28-9eca-b860b348c6ee']},
  '9e3cd8a4-6c03-4f28-9eca-b860b348c6ee': {'id': '9e3cd8a4-6c03-4f28-9eca-b860b348c6ee',
   'message': {'id': '9e3cd8a4-6c03-

In [10]:
conversation = json_obj['state']['loaderData']['routes/share.$shareId.($action)']['serverResponse']['data']['linear_conversation']
conversation_distilled = []
for message in conversation:
    role = message.get('message', {}).get('author', {}).get('role', '')
    message_content = message.get('message', {}).get('content', {}).get('parts', [])
    text = message.get('message', {}).get('content', {}).get('text', '')
    if role:
        conversation_distilled.append({
            'role': role,
            'message': message_content,
            'text': text,
        })

conversation_distilled

[{'role': 'system', 'message': [''], 'text': ''},
 {'role': 'user',
  'message': ['Original custom instructions no longer available'],
  'text': ''},
 {'role': 'user',
  'message': ['summarize this paper for me\nhttps://dl.acm.org/doi/abs/10.1145/3600006.3613165\npay attention to all sections and subsections'],
  'text': ''},
 {'role': 'assistant', 'message': [], 'text': ''},
 {'role': 'assistant',
  'message': [],
  'text': 'search("summarize this paper for me\\nhttps://dl.acm.org/doi/abs/10.1145/3600006.3613165\\npay attention to all sections and subsections")'},
 {'role': 'tool',
  'message': ['The output of this plugin was redacted.'],
  'text': ''},
 {'role': 'assistant',
  'message': ['\ue203The paper "Efficient Memory Management for Large Language Model Serving with PagedAttention" addresses the challenges of serving large language models (LLMs) efficiently, focusing on memory management to enhance throughput.\ue204 \ue200cite\ue202turn0search0\ue201\ue206\n\n**Abstract:**\n\ue2

In [11]:
with open('tmp2.json', 'w') as f:
    json.dump(conversation_distilled, f)